##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Основы настройки: тензоры и операции

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/customization/basics"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/customization/basics.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/customization/basics.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/customization/basics.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

Это вводное руководство по TensorFlow, в котором показано, как:

* Импортировать необходимый пакет
* Создавать и использовать тензоры
* Использовать ускорение графического процессора
* Демонстрация `tf.data.Dataset`

## Импорт TensorFlow

Сначала импортируйте модуль `tensorflow`. Начиная с TensorFlow версии 2, активное выполнение включено по умолчанию. Это обеспечивает более интерактивный интерфейс для TensorFlow, детали которого мы обсудим гораздо позже.

In [ ]:
import tensorflow as tf

## Тензоры

Тензор - это многомерный массив. Подобно объектам `ndarray` из NumPy , объекты` tf.Tensor` имеют тип данных и размерность. Кроме того, `tf.Tensor`ы могут находиться в памяти ускорителя (например, в графическом процессоре). TensorFlow предлагает богатую библиотеку операций ([tf.add](https://www.tensorflow.org/api_docs/python/tf/add), [tf.matmul](https://www.tensorflow.org/api_docs/python/tf/matmul), [tf.linalg.inv](https://www.tensorflow.org/api_docs/python/tf/linalg/inv) и т. д.), которые принимают и возвращают объекты `tf.Tensor`. Эти операции автоматически преобразуют нативные типы Python в тензоры, например:


In [ ]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

# Перегрузка операторов также поддерживается
print(tf.square(2) + tf.square(3))

Каждый `tf.Tensor` имеет размерность(shape) и тип данных(dtype):

In [ ]:
x = tf.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

Наиболее очевидные различия между массивами NumPy и `tf.Tensor`ами:

1. Тензоры могут обрабатываться в памяти ускорителя(например, GPU, TPU).
2. Тензоры неизменяемы.

### Совместимость с NumPy

Преобразование между TensorFlow `tf.Tensor` и NumPy` ndarray` очень простое:

* Операции TensorFlow автоматически преобразуют ndarrays NumPy в Tensor.
* Операции NumPy автоматически преобразуют тензоры в ndarrays NumPy.

Тензоры можно явно преобразовать в `ndarrays` NumPy с использованием метода `.numpy()`. Эти преобразования обычно дешевы, поскольку массив и tf.Tensor по возможности совместно используют базовое представление памяти. Однако совместное использование базового представления не всегда возможно, поскольку tf.Tensor может размещаться в памяти графического процессора, в то время как массивы NumPy всегда поддерживаются памятью хоста, а преобразование включает копирование из графического процессора в память хоста.

In [ ]:
import numpy as np

ndarray = np.ones([3, 3])

print("Операции TensorFlow автоматически конвертируют ndarray в tf.Tensor")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("И операции NumPy автоматически конвертируют tf.Tensor в ndarray")
print(np.add(tensor, 1))

print("Метод .numpy() явно конвертирует tf.Tensor в numpy ndarray")
print(tensor.numpy())

## GPU ускорение

Многие операции TensorFlow ускоряются для вычислений с использованием графического процессора. Без каких-либо аннотаций TensorFlow автоматически решает, использовать ли для операции GPU или CPU - при необходимости копируя тензор между CPU и памятью GPU. Тензоры, создаваемые операцией, обычно поддерживаются памятью устройства, на котором выполнялась операция, например:

In [ ]:
x = tf.random.uniform([3, 3])

print("Доступен ли графический процессор: "),
print(tf.config.experimental.list_physical_devices("GPU"))

print("Тензор на GPU #0:  "),
print(x.device.endswith('GPU:0'))

### Имена устройств

Свойство `Tensor.device` предоставляет полное строковое имя устройства, на котором размещается содержимое тензора. Это имя кодирует многие детали, такие как идентификатор сетевого адреса хоста, на котором выполняется эта программа, и устройства внутри этого хоста. Это требуется для распределенного выполнения программы TensorFlow. Строка заканчивается на `GPU: <N>`, если тензор помещается на `N`-й графический процессор хоста.

### Явное размещение на устройстве

В TensorFlow *размещение* определяет устройство, на котором будут выполняться(размещаться) отдельные операции. Как уже упоминалось, при отсутствии явных указаний TensorFlow автоматически решает, на каком устройстве выполнять операцию, и при необходимости копирует тензоры на это устройство. Однако операции TensorFlow могут быть явно размещены на определенных устройствах с помощью диспетчера контекста `tf.device`, например:

In [ ]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.matmul(x, x)

  result = time.time()-start

  print("10 loops: {:0.2f}ms".format(1000*result))

# Принудительное выполнение на CPU
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random.uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# Принудительное выполнение на GPU #0 если устройство доступно
if tf.config.experimental.list_physical_devices("GPU"):
  print("On GPU:")
  with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

## Наборы данных(датасеты)

В этом разделе используется [API `tf.data.Dataset`](https://www.tensorflow.org/guide/datasets) для создания конвейера подачи данных в вашу модель. API `tf.data.Dataset` используется для построения эффективных, сложных конвейеров ввода из простых, многократно используемых частей, которые будут подавать пакеты данных в циклы обучения или оценки вашей модели.

### Создание датасета

Создайте *исходный* набор данных, используя одну из фабричных функций, например [`Dataset.from_tensors`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensors), [`Dataset.from_tensor_slices`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices) или с использованием объектов, которые читают из файлов, таких как [`TextLineDataset`](https://www.tensorflow.org/api_docs/python/tf/data/TextLineDataset) или [`TFRecordDataset`](https://www.tensorflow.org/api_docs/python/tf/data/TFRecordDataset). 
См. [Руководство по наборам данных TensorFlow](https://www.tensorflow.org/guide/datasets#reading_input_data) для получения дополнительной информации.

In [ ]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# Создаем CSV файл
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line 1
Line 2
Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)

### Применение трансформаций

Используйте функции преобразования данных, такие как [`map`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map), [`batch`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch), или [`shuffle`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle).

In [ ]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

### Итерация

Объекты `tf.data.Dataset` поддерживают итерацию для перебора записей:

In [ ]:
print('Elements of ds_tensors:')
for x in ds_tensors:
  print(x)

print('\nElements in ds_file:')
for x in ds_file:
  print(x)